In [53]:
import pandas as pd
import numpy as np
from h3 import h3

In [2]:
df = pd.read_csv("h3_agg.csv")

In [3]:
df = df.rename(columns={'code_18': "corine_landuse", "percipitation":"precipitation", "Varv": "soil_class"})

In [4]:
df.describe()

,elevation,corine_landuse,temperature,precipitation,population,soil_class
count,124767.000000,124767.000000,124767.000000,124767.000000,124767.000000,88776.000000
mean,39.184832,306.372871,-32.096019,487.100083,8.764933,6.606932
std,37.905403,149.353962,20.975767,278.413802,143.997517,6.853250
min,-0.686404,0.000000,-61.000000,0.000000,0.000000,0.000000
25%,0.256986,211.000000,-50.000000,564.000000,0.000000,1.000000
50%,33.173809,311.000000,-42.000000,621.000000,0.149468,4.000000
75%,62.688461,512.000000,-10.000000,661.000000,1.125973,11.000000
max,278.191096,523.000000,0.000000,783.000000,31795.673828,22.000000


In [5]:
df.sample(3)

,cell_id,elevation,corine_landuse,temperature,precipitation,population,soil_class
119050,881f6591e1fffff,15.499795,311,-39.0,695.0,0.03462,11.0
116712,881f65571dfffff,38.748594,311,-35.0,656.0,0.00000,NaN
104286,881f64871bfffff,0.000000,523,0.0,0.0,0.00000,NaN


In [6]:
def h3_all_parents(base_cell, lowest_res=1):
    res = h3.h3_get_resolution(base_cell)
    t_cell = base_cell
    parents = []
    while res > lowest_res:
        parent = h3.h3_to_parent(t_cell)
        parents.append(parent)
        t_cell = parent
        res = h3.h3_get_resolution(parent)
    return parents


In [7]:
df['parent_ids'] = df['cell_id'].apply(lambda x: ":".join(h3_all_parents(x, lowest_res=2)))

In [8]:
df[f'parent_id_8'] = df['cell_id']

In [9]:
df.sample(5)

,cell_id,elevation,corine_landuse,temperature,precipitation,population,soil_class,parent_ids,parent_id_8
65365,88113464e9fffff,73.915213,231,-53.0,578.0,1.690376,8.0,87113464effffff:86113464fffffff:85113467ffffff...,88113464e9fffff
102762,881f64355bfffff,0.000000,523,0.0,0.0,0.000000,NaN,871f64355ffffff:861f64357ffffff:851f6437ffffff...,881f64355bfffff
19377,88089ab989fffff,26.556912,311,-30.0,657.0,0.120124,4.0,87089ab98ffffff:86089ab9fffffff:85089abbffffff...,88089ab989fffff
62713,88113441dbfffff,33.256639,512,-49.0,655.0,0.000000,0.0,87113441dffffff:86113441fffffff:85113443ffffff...,88113441dbfffff
60496,8811342441fffff,136.527781,312,-55.0,639.0,0.088142,4.0,871134244ffffff:861134247ffffff:85113427ffffff...,8811342441fffff


In [10]:
parent_df = []

variable = "elevation"

for idx in range(7,1,-1):
    print(idx)
    
    df[f'parent_id_{idx}'] = df[f'parent_id_{idx+1}'].map(h3.h3_to_parent)
    
    t8 = df.groupby(f'parent_id_{idx}')[variable].describe()
    t8 = pd.DataFrame(t8).drop(
                        columns=["min","25%","50%","75%","max"]
                    ).rename(columns={
                        'mean':variable, "std": variable + "_" + "std", "count": variable + "_" + "count"
                    })
    t8 = t8.reset_index(drop=False)
    t8['cell_id'] = t8[f'parent_id_{idx}']
    t8['resolution'] = idx
    t8['parent_ids'] = t8['cell_id'].apply(lambda x: ":".join(h3_all_parents(x, lowest_res=2)))
    t8 = t8[['cell_id', 'elevation', 'resolution', 'parent_ids', 'elevation_count','elevation_std']]
    t8['elevation_count'] = pd.to_numeric(t8['elevation_count'], downcast='integer')
    parent_df.append(t8)

7
6
5
4
3
2


In [11]:
subs = pd.concat(parent_df)

In [12]:
subs.sample(5)

,cell_id,elevation,resolution,parent_ids,elevation_count,elevation_std
3953,87089b66affffff,2.360919,7,86089b66fffffff:85089b67fffffff:84089b7fffffff...,7,2.493859
10056,8711349a5ffffff,46.283710,7,8611349a7ffffff:8511349bfffffff:8411349fffffff...,7,0.926130
8531,8711340deffffff,74.030782,7,8611340dfffffff:8511340ffffffff:8411341fffffff...,7,1.807254
12079,871135d56ffffff,83.482802,7,861135d57ffffff:851135d7fffffff:841135dfffffff...,7,1.018790
17984,871f65d9bffffff,0.000000,7,861f65d9fffffff:851f65dbfffffff:841f65dfffffff...,7,0.000000


In [13]:
subs.describe()

,elevation,resolution,elevation_count,elevation_std
count,21168.000000,21168.000000,21168.000000,21106.000000
mean,38.887971,6.822610,35.364796,2.524031
std,37.777596,0.465917,594.947494,3.579622
min,-0.000071,2.000000,1.000000,0.000000
25%,0.982977,7.000000,7.000000,0.023043
50%,32.890419,7.000000,7.000000,1.604654
75%,62.198756,7.000000,7.000000,3.211372
max,259.852890,7.000000,56885.000000,97.862109


In [14]:
elev = df[['cell_id','elevation', 'parent_ids']].copy()
elev['resolution']  = 8
elev['elevation_count']  = 1
elev['elevation_std']  = 0
elev = pd.concat([elev, subs])
elev['elevation_count'] = pd.to_numeric(elev['elevation_count'], downcast='integer')

In [15]:
elev.sample(5)

,cell_id,elevation,parent_ids,resolution,elevation_count,elevation_std
84030,881135da21fffff,96.697370,871135da2ffffff:861135da7ffffff:851135dbffffff...,8,1,0.000000
103852,881f64822bfffff,0.000000,871f64822ffffff:861f64827ffffff:851f6483ffffff...,8,1,0.000000
11936,871135c9bffffff,104.167037,861135c9fffffff:851135cbfffffff:841135dfffffff...,7,7,3.195369
6435,88089a0e6bfffff,19.162474,87089a0e6ffffff:86089a0e7ffffff:85089a0fffffff...,8,1,0.000000
19373,88089ab981fffff,23.069306,87089ab98ffffff:86089ab9fffffff:85089abbffffff...,8,1,0.000000


In [16]:
elev.describe()

,elevation,resolution,elevation_count,elevation_std
count,145935.000000,145935.000000,145935.000000,145873.000000
mean,39.141772,7.829218,5.984644,0.365196
std,37.886907,0.450996,226.907363,1.625518
min,-0.686404,2.000000,1.000000,0.000000
25%,0.396949,8.000000,1.000000,0.000000
50%,33.134811,8.000000,1.000000,0.000000
75%,62.632599,8.000000,1.000000,0.000000
max,278.191096,8.000000,56885.000000,97.862109


In [17]:
elev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145935 entries, 0 to 4
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   cell_id          145935 non-null  object 
 1   elevation        145935 non-null  float64
 2   parent_ids       145935 non-null  object 
 3   resolution       145935 non-null  int64  
 4   elevation_count  145935 non-null  int32  
 5   elevation_std    145873 non-null  float64
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 7.2+ MB


In [54]:
import sqlite3

conn = sqlite3.connect('test.db')
c = conn.cursor()

In [19]:
rs = c.execute("select * from dggs_catalog;")

In [20]:
for row in rs:
    print(row)

('srtm_30m_estonia_h3', 'H3', '24,58,27,59', 9, '9:8:7:6:5:4:3:2', 'elevation', '30m SRTM sampled at res 9 covering Estonia', None, 1634280469, None)


In [21]:
rs = c.execute("select * from srtm_30m_estonia_h3;")

In [22]:
for row in rs:
    print(row)

In [23]:
elev.to_sql('srtm_30m_estonia_h3', conn, if_exists='append', index = False)

In [24]:
c.close()

In [25]:
conn.close()

In [28]:
import yaml

In [29]:
fn = "../dggs_api_server/swagger/swagger-0.0.4.yaml"

In [34]:
with open(fn) as fh:
    read_data = yaml.load(fh, Loader=yaml.FullLoader)

# Print YAML data before sorting
# print(read_data)

dict_keys(['/', '/conformance', '/collections', '/collections/{collectionId}/describe', '/collections/{collectionId}/zones', '/collections/{collectionId}/zone', '/collections/{collectionId}/zone/{zoneId}/buffer', '/collections/{collectionId}/zone/{zoneId}/child', '/collections/{collectionId}/zone/{zoneId}/childOf', '/collections/{collectionId}/zone/{zoneId}/parentOf', '/collections/{collectionId}/zone/{zoneId}/siblingOf', '/collections/{collectionId}/zone/{zoneId}/intersects', '/collections/{collectionId}/zone/{zoneId}/overlaps'])

In [50]:
reorder = {}

for k, v in read_data['paths'].items():
    # print(k)
    for method in ['get', 'post', 'options', 'put', 'delete', 'head']:
        # print(v['x-openapi-router-controller'])
        if method in v.keys():
            path = v[method]['x-openapi-router-controller']
            print(f"{k} [{method}]: {path}")
            reorder.update( { f"{k} [{method}]": path} )

/ [get]: dggs_api_server.controllers.capabilities_controller
/conformance [get]: dggs_api_server.controllers.capabilities_controller
/collections [get]: dggs_api_server.controllers.capabilities_controller
/collections/{collectionId}/describe [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zones [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/buffer [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/child [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/childOf [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/parentOf [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/siblingOf [get]: dggs_api_server.controllers.

In [45]:
fn2 = "../dggs_api_server/swagger/swagger-0.0.6.yaml"

In [46]:
with open(fn2) as fh:
    read_data2 = yaml.load(fh, Loader=yaml.FullLoader)

In [52]:
reorder2 = {}

for k, v in read_data2['paths'].items():
    # print(k)
    for method in ['get', 'post', 'options', 'put', 'delete', 'head']:
        # print(v['x-openapi-router-controller'])
        if method in v.keys():
            path = v[method]['x-openapi-router-controller'].replace("<<","").replace(">>", "_")
            print(f"{k} [{method}]: {path}")
            reorder2.update( { f"{k} [{method}]": path} )

/ [get]: dggs_api_server.controllers.capabilities_controller
/conformance [get]: dggs_api_server.controllers.capabilities_controller
/collections [get]: dggs_api_server.controllers.capabilities_controller
/collections/{collectionId}/describe [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zones [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/distance [get]: dggs_api_server.controllers.zone_query_ops_controller
/collections/{collectionId}/zone/{zoneId}/buffer [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/difference [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/intersection [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/symDifference [get]: dg

In [55]:
import sqlite3

conn = sqlite3.connect('test.db')
c = conn.cursor()

In [56]:
rs = conn.execute(
        "SELECT table_name, dggs_type, resolutions, variables, description, meta_url FROM dggs_catalog"
    )
    

In [57]:
rs.description

(('table_name', None, None, None, None, None, None),
 ('dggs_type', None, None, None, None, None, None),
 ('resolutions', None, None, None, None, None, None),
 ('variables', None, None, None, None, None, None),
 ('description', None, None, None, None, None, None),
 ('meta_url', None, None, None, None, None, None))

In [61]:
rs

In [62]:
c_list = []
for row in rs:
    links = [row[5]] if row[5] is not None else []
    resolutions = row[2].split(":") if row[2] is not None else []

    c = {'id':row[0],
        'dggs_id':row[1],
        'title':row[0],
        'description':row[4],
        'resolutions':resolutions,
        'links':links,
        }
    c_list.append(c)

In [63]:
c_list

[{'id': 'srtm_30m_estonia_h3',
  'dggs_id': 'H3',
  'title': 'srtm_30m_estonia_h3',
  'description': '30m SRTM sampled at res 8 covering Estonia',
  'resolutions': ['8', '7', '6', '5', '4', '3', '2'],
  'links': []}]

In [65]:
collection_id = c_list[0]['id']

rs = conn.execute(
        "SELECT * FROM {} limit 1".format(
            collection_id
        )
    )
names = [description[0] for description in rs.description]

In [67]:
that_row = rs.fetchone()

In [66]:
names

['cell_id',
 'elevation',
 'resolution',
 'parent_ids',
 'elevation_count',
 'elevation_std']

In [74]:
not_parent_ids_data = list(filter(lambda nt: nt[0] not in ['parent_ids'], zip(names, that_row)))
all_fields = list(zip(names, that_row))

In [75]:
not_parent_ids_data

[('cell_id', '8808932091fffff'),
 ('elevation', 0.0),
 ('resolution', 8),
 ('elevation_count', 1),
 ('elevation_std', 0.0)]

In [76]:
all_fields = dict(zip(names, that_row))

In [77]:
all_fields

{'cell_id': '8808932091fffff',
 'elevation': 0.0,
 'resolution': 8,
 'parent_ids': '870893209ffffff:86089320fffffff:85089323fffffff:8408933ffffffff:830893fffffffff:820897fffffffff',
 'elevation_count': 1,
 'elevation_std': 0.0}

In [80]:
[print({k: v}) for (k, v) in all_fields.items() if k not in ['cell_id', 'parent_ids'] ]

{'elevation': 0.0}
{'resolution': 8}
{'elevation_count': 1}
{'elevation_std': 0.0}


[None, None, None, None]

In [81]:
cell_id_name="cell_id"
ignore_fields=["parent_ids"]

In [83]:
[cell_id_name] + ignore_fields

['cell_id', 'parent_ids']

In [89]:
import folium

In [91]:
def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [92]:
def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(height=600, width=800, location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m

In [99]:
minx, miny, maxx, maxy = [25.32, 58.27, 26.82, 58.67]
resolution = 5

area_json = {'type': 'Polygon',
            'coordinates': [
                [
                    [miny, minx],
                    [miny, maxx],
                    [maxy, maxx],
                    [maxy, minx],
                    [miny, minx]
                ]
            ]
        }
zone_id_list_from_fill = h3.polyfill(area_json, resolution)

In [100]:
len(zone_id_list_from_fill)

19

In [111]:
h3.h3_get_resolution("85089323fffffff")

5

In [112]:
"85089323fffffff" in zone_id_list_from_fill

False

In [103]:
zone_id_list_from_fill

{'8511340bfffffff',
 '8511341bfffffff',
 '85113443fffffff',
 '85113447fffffff',
 '8511344ffffffff',
 '85113453fffffff',
 '85113457fffffff',
 '85113463fffffff',
 '8511346bfffffff',
 '8511346ffffffff',
 '85113473fffffff',
 '8511347bfffffff',
 '851134c3fffffff',
 '851134c7fffffff',
 '851134cbfffffff',
 '851134cffffffff',
 '851134dbfffffff',
 '851136a7fffffff',
 '851136b7fffffff'}

In [101]:


m = visualize_hexagons(zone_id_list_from_fill)
display(m)



In [110]:
len(tf['features'])

100

In [109]:
tf = {
  "features": [
    {
      "type": {
        "geometry": [
          "85089323fffffff"
        ],
        "id": "85089323fffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 26,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089327fffffff"
        ],
        "id": "85089327fffffff",
        "properties": {
          "elevation": 2.7734900354328675,
          "elevation_count": 293,
          "elevation_std": 3.3743283726767057,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508932bfffffff"
        ],
        "id": "8508932bfffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 100,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508932ffffffff"
        ],
        "id": "8508932ffffffff",
        "properties": {
          "elevation": 1.0719031220642885,
          "elevation_count": 343,
          "elevation_std": 3.141205783613213,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089367fffffff"
        ],
        "id": "85089367fffffff",
        "properties": {
          "elevation": 5.7584671210184775,
          "elevation_count": 192,
          "elevation_std": 7.575306948078423,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508936ffffffff"
        ],
        "id": "8508936ffffffff",
        "properties": {
          "elevation": 0.00034633265112951606,
          "elevation_count": 62,
          "elevation_std": 0.002727026022021195,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508980bfffffff"
        ],
        "id": "8508980bfffffff",
        "properties": {
          "elevation": 0.809628440832385,
          "elevation_count": 53,
          "elevation_std": 1.9227926820761838,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089843fffffff"
        ],
        "id": "85089843fffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 343,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089847fffffff"
        ],
        "id": "85089847fffffff",
        "properties": {
          "elevation": 1.2554001654013693,
          "elevation_count": 343,
          "elevation_std": 2.9348974035552757,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508984bfffffff"
        ],
        "id": "8508984bfffffff",
        "properties": {
          "elevation": 2.5848036925055724,
          "elevation_count": 343,
          "elevation_std": 2.640274392113855,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508984ffffffff"
        ],
        "id": "8508984ffffffff",
        "properties": {
          "elevation": 11.957274671028115,
          "elevation_count": 343,
          "elevation_std": 6.596485257743632,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089853fffffff"
        ],
        "id": "85089853fffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 343,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089857fffffff"
        ],
        "id": "85089857fffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 230,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508985bfffffff"
        ],
        "id": "8508985bfffffff",
        "properties": {
          "elevation": 2.7115445549648656,
          "elevation_count": 343,
          "elevation_std": 3.96034583128291,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089863fffffff"
        ],
        "id": "85089863fffffff",
        "properties": {
          "elevation": 0.5740724141824757,
          "elevation_count": 314,
          "elevation_std": 1.9472336573537568,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089867fffffff"
        ],
        "id": "85089867fffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 53,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508986bfffffff"
        ],
        "id": "8508986bfffffff",
        "properties": {
          "elevation": 8.024075038040433,
          "elevation_count": 343,
          "elevation_std": 6.997358832234688,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508986ffffffff"
        ],
        "id": "8508986ffffffff",
        "properties": {
          "elevation": 4.066422573119096,
          "elevation_count": 343,
          "elevation_std": 7.664175999187078,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089873fffffff"
        ],
        "id": "85089873fffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 197,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "8508987bfffffff"
        ],
        "id": "8508987bfffffff",
        "properties": {
          "elevation": 7.9407373937326735,
          "elevation_count": 343,
          "elevation_std": 7.170736162403771,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "850898cbfffffff"
        ],
        "id": "850898cbfffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 188,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "850898cffffffff"
        ],
        "id": "850898cffffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 23,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "850898dbfffffff"
        ],
        "id": "850898dbfffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 2,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a03fffffff"
        ],
        "id": "85089a03fffffff",
        "properties": {
          "elevation": 12.922876119266098,
          "elevation_count": 343,
          "elevation_std": 4.808862979315825,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a07fffffff"
        ],
        "id": "85089a07fffffff",
        "properties": {
          "elevation": 16.648168794742947,
          "elevation_count": 343,
          "elevation_std": 4.446743224740138,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a0bfffffff"
        ],
        "id": "85089a0bfffffff",
        "properties": {
          "elevation": 20.399990749478288,
          "elevation_count": 343,
          "elevation_std": 3.221455154543812,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a0ffffffff"
        ],
        "id": "85089a0ffffffff",
        "properties": {
          "elevation": 21.674543681870734,
          "elevation_count": 343,
          "elevation_std": 4.661982794238665,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a13fffffff"
        ],
        "id": "85089a13fffffff",
        "properties": {
          "elevation": 11.97491595031831,
          "elevation_count": 343,
          "elevation_std": 4.738955259953936,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a17fffffff"
        ],
        "id": "85089a17fffffff",
        "properties": {
          "elevation": 10.040117488707764,
          "elevation_count": 343,
          "elevation_std": 4.166940244753431,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a1bfffffff"
        ],
        "id": "85089a1bfffffff",
        "properties": {
          "elevation": 19.957476758500132,
          "elevation_count": 343,
          "elevation_std": 3.4668060762024235,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a23fffffff"
        ],
        "id": "85089a23fffffff",
        "properties": {
          "elevation": 48.08325914741662,
          "elevation_count": 343,
          "elevation_std": 6.633097076311552,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a27fffffff"
        ],
        "id": "85089a27fffffff",
        "properties": {
          "elevation": 64.19822113165121,
          "elevation_count": 343,
          "elevation_std": 4.784722653334324,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a2bfffffff"
        ],
        "id": "85089a2bfffffff",
        "properties": {
          "elevation": 51.05603462355875,
          "elevation_count": 343,
          "elevation_std": 7.1762449786842675,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a2ffffffff"
        ],
        "id": "85089a2ffffffff",
        "properties": {
          "elevation": 66.24361959414159,
          "elevation_count": 343,
          "elevation_std": 5.322151829644917,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a33fffffff"
        ],
        "id": "85089a33fffffff",
        "properties": {
          "elevation": 36.094246885937814,
          "elevation_count": 343,
          "elevation_std": 5.4974660068692955,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a37fffffff"
        ],
        "id": "85089a37fffffff",
        "properties": {
          "elevation": 53.01072062596038,
          "elevation_count": 343,
          "elevation_std": 5.935331869352657,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a3bfffffff"
        ],
        "id": "85089a3bfffffff",
        "properties": {
          "elevation": 30.088826663689733,
          "elevation_count": 343,
          "elevation_std": 4.518931424509177,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a43fffffff"
        ],
        "id": "85089a43fffffff",
        "properties": {
          "elevation": 12.781450793178548,
          "elevation_count": 343,
          "elevation_std": 10.72027082983594,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a47fffffff"
        ],
        "id": "85089a47fffffff",
        "properties": {
          "elevation": 12.2124409365854,
          "elevation_count": 343,
          "elevation_std": 3.634476216273337,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a4bfffffff"
        ],
        "id": "85089a4bfffffff",
        "properties": {
          "elevation": 0.48804777796738424,
          "elevation_count": 343,
          "elevation_std": 2.1310724984532965,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a4ffffffff"
        ],
        "id": "85089a4ffffffff",
        "properties": {
          "elevation": 3.9759275554966096,
          "elevation_count": 343,
          "elevation_std": 4.959636091583174,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a53fffffff"
        ],
        "id": "85089a53fffffff",
        "properties": {
          "elevation": 17.089578025198666,
          "elevation_count": 343,
          "elevation_std": 11.198183840639508,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a57fffffff"
        ],
        "id": "85089a57fffffff",
        "properties": {
          "elevation": 21.050904862039634,
          "elevation_count": 343,
          "elevation_std": 3.291827547738877,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a5bfffffff"
        ],
        "id": "85089a5bfffffff",
        "properties": {
          "elevation": 2.862631676632024,
          "elevation_count": 343,
          "elevation_std": 5.736414618301653,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a63fffffff"
        ],
        "id": "85089a63fffffff",
        "properties": {
          "elevation": 29.198125665430553,
          "elevation_count": 343,
          "elevation_std": 4.68715142971771,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a67fffffff"
        ],
        "id": "85089a67fffffff",
        "properties": {
          "elevation": 45.317935878122455,
          "elevation_count": 343,
          "elevation_std": 5.9746588804289,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a6bfffffff"
        ],
        "id": "85089a6bfffffff",
        "properties": {
          "elevation": 23.439003525635954,
          "elevation_count": 343,
          "elevation_std": 2.974304840782703,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a6ffffffff"
        ],
        "id": "85089a6ffffffff",
        "properties": {
          "elevation": 28.450586900582255,
          "elevation_count": 343,
          "elevation_std": 2.412782639590598,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a73fffffff"
        ],
        "id": "85089a73fffffff",
        "properties": {
          "elevation": 20.876544943767687,
          "elevation_count": 343,
          "elevation_std": 4.14608082697055,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a77fffffff"
        ],
        "id": "85089a77fffffff",
        "properties": {
          "elevation": 36.0824210798534,
          "elevation_count": 343,
          "elevation_std": 5.898193559605743,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a7bfffffff"
        ],
        "id": "85089a7bfffffff",
        "properties": {
          "elevation": 16.941097272359517,
          "elevation_count": 343,
          "elevation_std": 3.8721929269819952,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a83fffffff"
        ],
        "id": "85089a83fffffff",
        "properties": {
          "elevation": 1.3279570965483758,
          "elevation_count": 343,
          "elevation_std": 2.2514678774715198,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a87fffffff"
        ],
        "id": "85089a87fffffff",
        "properties": {
          "elevation": 6.954967837598406,
          "elevation_count": 343,
          "elevation_std": 3.6001624368942595,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a8bfffffff"
        ],
        "id": "85089a8bfffffff",
        "properties": {
          "elevation": 5.617733538587662,
          "elevation_count": 343,
          "elevation_std": 5.349310128037432,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a8ffffffff"
        ],
        "id": "85089a8ffffffff",
        "properties": {
          "elevation": 5.294141926909143,
          "elevation_count": 343,
          "elevation_std": 3.7345098189955577,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a93fffffff"
        ],
        "id": "85089a93fffffff",
        "properties": {
          "elevation": 0.2759482992153704,
          "elevation_count": 343,
          "elevation_std": 1.2312969064764312,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a97fffffff"
        ],
        "id": "85089a97fffffff",
        "properties": {
          "elevation": 0.46903840923512835,
          "elevation_count": 343,
          "elevation_std": 1.4448293509896308,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089a9bfffffff"
        ],
        "id": "85089a9bfffffff",
        "properties": {
          "elevation": 1.859097681044963,
          "elevation_count": 343,
          "elevation_std": 4.622741647979972,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089aa3fffffff"
        ],
        "id": "85089aa3fffffff",
        "properties": {
          "elevation": 35.86852392937828,
          "elevation_count": 343,
          "elevation_std": 6.837064279276079,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089aa7fffffff"
        ],
        "id": "85089aa7fffffff",
        "properties": {
          "elevation": 29.92106214531146,
          "elevation_count": 343,
          "elevation_std": 8.27319224565434,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089aabfffffff"
        ],
        "id": "85089aabfffffff",
        "properties": {
          "elevation": 30.274659216095422,
          "elevation_count": 343,
          "elevation_std": 6.3882138848460635,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089aaffffffff"
        ],
        "id": "85089aaffffffff",
        "properties": {
          "elevation": 42.625494130626485,
          "elevation_count": 343,
          "elevation_std": 5.007586902485049,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089ab3fffffff"
        ],
        "id": "85089ab3fffffff",
        "properties": {
          "elevation": 15.927725438964163,
          "elevation_count": 343,
          "elevation_std": 8.174357283062427,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089ab7fffffff"
        ],
        "id": "85089ab7fffffff",
        "properties": {
          "elevation": 18.619015731965867,
          "elevation_count": 343,
          "elevation_std": 3.4261837953098007,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089abbfffffff"
        ],
        "id": "85089abbfffffff",
        "properties": {
          "elevation": 20.92643241592369,
          "elevation_count": 343,
          "elevation_std": 7.5971599486455155,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089ac3fffffff"
        ],
        "id": "85089ac3fffffff",
        "properties": {
          "elevation": 0.2661438170937631,
          "elevation_count": 343,
          "elevation_std": 1.4448099289939111,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089ac7fffffff"
        ],
        "id": "85089ac7fffffff",
        "properties": {
          "elevation": 10.589398591522334,
          "elevation_count": 343,
          "elevation_std": 6.030157019242921,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089acbfffffff"
        ],
        "id": "85089acbfffffff",
        "properties": {
          "elevation": 2.628989255483738,
          "elevation_count": 343,
          "elevation_std": 5.693886181415954,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089acffffffff"
        ],
        "id": "85089acffffffff",
        "properties": {
          "elevation": 20.36868843862698,
          "elevation_count": 343,
          "elevation_std": 9.611026172025525,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089ad3fffffff"
        ],
        "id": "85089ad3fffffff",
        "properties": {
          "elevation": 1.158985061607582,
          "elevation_count": 343,
          "elevation_std": 1.6683976676849666,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089ad7fffffff"
        ],
        "id": "85089ad7fffffff",
        "properties": {
          "elevation": 2.8653559388367364,
          "elevation_count": 343,
          "elevation_std": 4.107440140546029,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089adbfffffff"
        ],
        "id": "85089adbfffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 343,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b03fffffff"
        ],
        "id": "85089b03fffffff",
        "properties": {
          "elevation": 2.63216290964525,
          "elevation_count": 343,
          "elevation_std": 7.69013399793049,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b07fffffff"
        ],
        "id": "85089b07fffffff",
        "properties": {
          "elevation": 0.06890933167363156,
          "elevation_count": 169,
          "elevation_std": 0.41990766775625693,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b0bfffffff"
        ],
        "id": "85089b0bfffffff",
        "properties": {
          "elevation": 27.893969744307686,
          "elevation_count": 343,
          "elevation_std": 15.84406581485137,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b0ffffffff"
        ],
        "id": "85089b0ffffffff",
        "properties": {
          "elevation": 0.22782027361166554,
          "elevation_count": 343,
          "elevation_std": 1.1821700197964602,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b13fffffff"
        ],
        "id": "85089b13fffffff",
        "properties": {
          "elevation": 0.6392720099897466,
          "elevation_count": 343,
          "elevation_std": 2.4413918273261404,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b17fffffff"
        ],
        "id": "85089b17fffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 104,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b1bfffffff"
        ],
        "id": "85089b1bfffffff",
        "properties": {
          "elevation": 27.003739540513667,
          "elevation_count": 343,
          "elevation_std": 16.718706153326185,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b2bfffffff"
        ],
        "id": "85089b2bfffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 215,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b3bfffffff"
        ],
        "id": "85089b3bfffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 195,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b43fffffff"
        ],
        "id": "85089b43fffffff",
        "properties": {
          "elevation": 59.01608313635783,
          "elevation_count": 343,
          "elevation_std": 6.718984798817579,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b47fffffff"
        ],
        "id": "85089b47fffffff",
        "properties": {
          "elevation": 45.58926126862889,
          "elevation_count": 343,
          "elevation_std": 5.481334422373411,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b4bfffffff"
        ],
        "id": "85089b4bfffffff",
        "properties": {
          "elevation": 75.77274557583364,
          "elevation_count": 343,
          "elevation_std": 3.61067671217621,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b4ffffffff"
        ],
        "id": "85089b4ffffffff",
        "properties": {
          "elevation": 66.70132977219022,
          "elevation_count": 343,
          "elevation_std": 7.336595758674981,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b53fffffff"
        ],
        "id": "85089b53fffffff",
        "properties": {
          "elevation": 60.55847303515677,
          "elevation_count": 343,
          "elevation_std": 7.327268183773321,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b57fffffff"
        ],
        "id": "85089b57fffffff",
        "properties": {
          "elevation": 43.29900223266761,
          "elevation_count": 343,
          "elevation_std": 3.5809538782289407,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b5bfffffff"
        ],
        "id": "85089b5bfffffff",
        "properties": {
          "elevation": 70.20181345682876,
          "elevation_count": 343,
          "elevation_std": 3.1461800734609766,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b63fffffff"
        ],
        "id": "85089b63fffffff",
        "properties": {
          "elevation": 39.36971203013816,
          "elevation_count": 343,
          "elevation_std": 19.518738999459437,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b67fffffff"
        ],
        "id": "85089b67fffffff",
        "properties": {
          "elevation": 6.538035606835203,
          "elevation_count": 343,
          "elevation_std": 9.042243471074084,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b6bfffffff"
        ],
        "id": "85089b6bfffffff",
        "properties": {
          "elevation": 74.66082297700942,
          "elevation_count": 343,
          "elevation_std": 4.659413427986922,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b6ffffffff"
        ],
        "id": "85089b6ffffffff",
        "properties": {
          "elevation": 54.78849101824813,
          "elevation_count": 343,
          "elevation_std": 18.19405834446748,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b73fffffff"
        ],
        "id": "85089b73fffffff",
        "properties": {
          "elevation": 25.587492567064757,
          "elevation_count": 343,
          "elevation_std": 18.03647803300845,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b77fffffff"
        ],
        "id": "85089b77fffffff",
        "properties": {
          "elevation": 1.8572071050025825,
          "elevation_count": 343,
          "elevation_std": 5.5947193980443135,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b7bfffffff"
        ],
        "id": "85089b7bfffffff",
        "properties": {
          "elevation": 59.64353820556561,
          "elevation_count": 343,
          "elevation_std": 6.7081582012479934,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b8bfffffff"
        ],
        "id": "85089b8bfffffff",
        "properties": {
          "elevation": 2.362284053606798,
          "elevation_count": 299,
          "elevation_std": 6.689338501646519,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b8ffffffff"
        ],
        "id": "85089b8ffffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 24,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089b9bfffffff"
        ],
        "id": "85089b9bfffffff",
        "properties": {
          "elevation": 0,
          "elevation_count": 174,
          "elevation_std": 0,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089bc3fffffff"
        ],
        "id": "85089bc3fffffff",
        "properties": {
          "elevation": 32.8613869562136,
          "elevation_count": 343,
          "elevation_std": 4.65675288608828,
          "resolution": 5
        },
        "type": "Feature"
      }
    },
    {
      "type": {
        "geometry": [
          "85089bc7fffffff"
        ],
        "id": "85089bc7fffffff",
        "properties": {
          "elevation": 18.83016864623181,
          "elevation_count": 343,
          "elevation_std": 16.053327075864214,
          "resolution": 5
        },
        "type": "Feature"
      }
    }
  ],
  "type": "FeatureCollection"
}